In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
from sqlalchemy import create_engine
import re
from MTIPmapping import *

In [2]:
path = r'T:\MPO\TIP\TIP FY24-27\Maps\Data'

In [3]:
file = path + '\\MTIP24_27_List.xlsx'

In [4]:
data = pd.read_excel(file)

In [5]:
data.columns

Index(['Geo', 'Project Name', 'MTIP ID #', 'Project Description', 'Work Type',
       'Perf. Meas.', 'RTP Project Number / Ref.', 'In AQ CATS?', 'In\nAQMA?',
       'Air Quality Status', 'STIP Key', 'FFY', 'Phase', 'Phase Status',
       'Federal Funding ', 'Federal Funding Source', 'Federal Req. Match',
       'Federal Req. Match Source', 'Total Fed+ Req Match', 'Other Funding',
       'Other Funding Source', 'Total All Sources'],
      dtype='object')

In [6]:
data['Air Quality Status'].unique()

array(['EXEMPT / Other-Planning and Technical Studies (IAC conf 4/26/20)',
       nan, 'EXEMPT / Other-Planning and Technical Studies',
       'N/A (IAC conf 4/26/20)', 'N/A (IAC conf 11/3/21)',
       'EXEMPT / Air Quality - Bicycle and Pedestrian facilities',
       'Outside PM10 air quality maintenance area',
       'Outside PM10 air quality maintenance area (IAC conf 11/3/21)',
       'EXEMPT / Safety - Highway Safety Improvement Program implementation',
       'EXEMPT / Other - Specific activities which do not involve or lead directly to construction',
       'EXEMPT / Other - Planning and Technical Studies',
       'EXEMPT / Safety - Projects that correct, improve, or eliminate a hazardous location or feature',
       'Part of K16223, project level conformity was approved in interagency meeting 12/23/2020',
       'EXEMPT / Safety - Highway Safety Improvement Program implementation; Air Quality - Bicycle and Pedestrian facilities (IAC conf 4/26/20)',
       'EXEMPT / Safety - Pro

In [8]:
data['AQ Exempt?'] = data['Air Quality Status'].apply(lambda x: getAQvars(x, var='AQ Exempt?')) 

In [9]:
data['AQ Status'] = data['Air Quality Status'].apply(lambda x: getAQvars(x, var='AQ Status')) 

In [10]:
data['IAC'] = data['Air Quality Status'].apply(lambda x: getAQvars(x, var='IAC')) 

In [11]:
# this data has been modified in ArcGIS Pro after mapping
points = gpd.read_file(os.path.join(path, 'FY24_27_points.shp'))

In [12]:
# this data has been modified in ArcGIS Pro after mapping
lines = gpd.read_file(os.path.join(path, 'FY24_27_lines.shp'))

In [25]:
pointdata = get_district_IDs(dat=newpts, shptype='point', keycol='STIP_Key', 
                             colnm1='Senator District', colnm2='Representative District')

In [26]:
linedata = get_district_IDs(dat=lines, shptype='line')

In [27]:
df = pointdata.append(linedata , ignore_index=True)

In [28]:
df.rename(columns={'STIP_Key':'STIP Key'}, inplace=True)

In [29]:
df['STIP Key'] = df['STIP Key'].astype(np.int64)

In [30]:
data.shape

(180, 25)

In [31]:
ndata = data.merge(df, how='outer', on='STIP Key')

In [32]:
ndata.shape

(188, 27)

In [33]:
ndata.to_excel(os.path.join(path, 'MTIP24_27_List_Updated.xlsx'), 
               sheet_name='2024-2027 Project List', 
               index=False)  